In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import re

## Read in All Data

In [66]:
source_dir = '/mnt/c/Users/natra/Documents/Research/Iraq-post-conflict-rebel-governance'
iom_adm3_mapping = source_dir + '/data/iom/iom_adm3_mapping.csv'
iom_data_file = source_dir + '/data/iom/iom_raw.xlsx'
ag_indices_file = source_dir + '/data/satellite_indices/evi/evi_mask_mean_max_min/evi_stats_Jan2001-Dec2022.csv'
adm3_xsub_file = source_dir + '/data/conflict/adm3_xsub_monthly.csv'
pop_dir = source_dir + '/data/population/'
# NOTE - as-is, there is a duplicate ADM3_PCODE in this shapefile!! 
# IQG01Q05N01 is for both Abi Gharaq and Markaz Ana
iraq_shp_adm3_loc = source_dir + '/data/geo_boundaries/irq-administrative-divisions-shapefiles/irq_admbnda_adm3_cso_20190603.shp'

In [4]:
iom_adm3_map = pd.read_csv(iom_adm3_mapping)
iom_data = pd.read_excel(iom_data_file, sheet_name="Full Dataset",header=1)
ag_data = pd.read_csv(ag_indices_file)
adm3_xsub = pd.read_csv(adm3_xsub_file)
iraq_shp_adm3 = gpd.read_file(iraq_shp_adm3_loc)

In [59]:
# obtain desired info from IOM
iom_data_filt = iom_data.loc[:,('LocationID','DateofInterview','Q1.10 IsLocationInDisputedArea',
                                'Q1.11 WasLocationOccupied','Q1.11.1 RetakenPeriod',
                                'Q1.12 FirstMajorEthno-religiousPrior','Q1.13 FirstMajorEthno-religiousCurrent')]

In [49]:
# concatenate population year files
# processing of global tif files into iraq adm3 regions done in Google Colab Pro - 
# below I'm reading in the output of the Colab files

pop_dfs = []
for file in os.listdir(pop_dir):
    if file.endswith('.csv'):
        file_path = pop_dir + file
        df = pd.read_csv(file_path, index_col=None)
        pop_dfs.append(df)
pop_df_full = pd.concat(pop_dfs, axis=0, ignore_index=True)

In [50]:
pop_df_full

,ADM3_EN,Shape_Area,year,pop_count,pop_density
0,Abi Gharaq,0.015754,2000,137199.660,8.709119e+06
1,Abu Dalf,0.145883,2000,17110.668,1.172901e+05
2,Abu Ghraib,0.064395,2000,603092.300,9.365519e+06
3,Agjalare,0.068020,2000,43407.895,6.381599e+05
4,Akaika,0.025196,2000,30088.568,1.194168e+06
...,...,...,...,...,...
1465,Wasit,0.196401,2020,229746.940,1.169786e+06
1466,Yaijy,0.009094,2020,39935.473,4.391432e+06
1467,Yathreb,0.045412,2020,50930.043,1.121499e+06
1468,Zawita,0.048012,2020,261342.340,5.443227e+06


## Merge all data at monthly, adm3 level

In [35]:
# get max evi at monthly level
ag_data_max = ag_data.loc[:,('ADM3_EN','max_evi','max_evi_scaled','date')]
ag_data_max['date'] = pd.to_datetime(ag_data_max['date'])
ag_data_max = ag_data_max.set_index('date')
ag_data_max_month = ag_data_max.groupby(['ADM3_EN'])[['max_evi','max_evi_scaled']].resample('ME').max()

ag_data_max_month.head()

max_evi  max_evi_scaled
ADM3_EN    date                               
Abi Gharaq 2001-01-31   5416.0          0.5416
           2001-02-28   6291.0          0.6291
           2001-03-31   6513.0          0.6513
           2001-04-30   5332.0          0.5332
           2001-05-31   5147.0          0.5147

In [36]:
# get mean evi at monthly level
ag_data_avg = ag_data.loc[:,('ADM3_EN','mean_evi','mean_evi_scaled','date')]
ag_data_avg['date'] = pd.to_datetime(ag_data_avg['date'])
ag_data_avg = ag_data_avg.set_index('date')
ag_data_avg_month = ag_data_avg.groupby(['ADM3_EN'])[['mean_evi','mean_evi_scaled']].resample('ME').mean()

ag_data_avg_month.head()

mean_evi  mean_evi_scaled
ADM3_EN    date                                    
Abi Gharaq 2001-01-31  2886.373490         0.288637
           2001-02-28  3094.206464         0.309421
           2001-03-31  3275.809481         0.327581
           2001-04-30  2596.503366         0.259650
           2001-05-31  2161.217474         0.216122

In [37]:
ag_data_monthly = ag_data_avg_month.join(ag_data_max_month)
print(ag_data_avg_month.shape)
print(ag_data_max_month.shape)
print(ag_data_monthly.shape)

(77616, 2)
(77616, 2)
(77616, 4)


In [38]:
# xsub only goes until 2019-12-31
adm3_xsub_mon = adm3_xsub.rename(columns={'month':'date'})
adm3_xsub_mon = adm3_xsub_mon.drop(columns=['ADM3_PCODE'])
adm3_xsub_mon = adm3_xsub_mon.set_index(['ADM3_EN','date'])
adm3_xsub_mon.head()

ACTION_IND  ACTION_DIR  ACTION_PRT
ADM3_EN    date                                          
Abi Gharaq 2017-12-31           1           1           0
           2018-01-31           0           0           0
           2018-02-28           0           0           0
           2018-03-31           0           0           0
           2018-04-30           0           0           0

In [39]:
# join agricultural EVI indices with xsub conflict data
adm3_iraq_data = ag_data_monthly.join(adm3_xsub_mon)
print(ag_data_monthly.shape)
print(adm3_xsub_mon.shape)
print(adm3_iraq_data.shape)
adm3_iraq_data = adm3_iraq_data.reset_index()

(77616, 4)
(23658, 3)
(77616, 7)


In [40]:
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT
0,Abi Gharaq,2001-01-31,2886.373490,0.288637,5416.0,0.5416,NaN,NaN,NaN
1,Abi Gharaq,2001-02-28,3094.206464,0.309421,6291.0,0.6291,NaN,NaN,NaN
2,Abi Gharaq,2001-03-31,3275.809481,0.327581,6513.0,0.6513,NaN,NaN,NaN
3,Abi Gharaq,2001-04-30,2596.503366,0.259650,5332.0,0.5332,NaN,NaN,NaN
4,Abi Gharaq,2001-05-31,2161.217474,0.216122,5147.0,0.5147,NaN,NaN,NaN


In [41]:
# no duplicates at this point
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['date','ADM3_EN'],keep=False)]

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT


In [42]:
# make xsub conflict values 0 for un-defined time periods where we have xsub data; otherwise 
# keep NA. Have data from 2001-01-01 until 2019-12-31
min_xsub = pd.to_datetime('2001-01-01')
max_xsub = adm3_xsub.month.max()

adm3_iraq_data['ACTION_IND'] = np.where(((adm3_iraq_data['date']>=min_xsub)&(adm3_iraq_data['date']<=max_xsub)&(adm3_iraq_data.ACTION_IND.isna())),0,adm3_iraq_data.ACTION_IND)
adm3_iraq_data['ACTION_DIR'] = np.where(((adm3_iraq_data['date']>=min_xsub)&(adm3_iraq_data['date']<=max_xsub)&(adm3_iraq_data.ACTION_DIR.isna())),0,adm3_iraq_data.ACTION_DIR)
adm3_iraq_data['ACTION_PRT'] = np.where(((adm3_iraq_data['date']>=min_xsub)&(adm3_iraq_data['date']<=max_xsub)&(adm3_iraq_data.ACTION_PRT.isna())),0,adm3_iraq_data.ACTION_PRT)
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT
0,Abi Gharaq,2001-01-31,2886.373490,0.288637,5416.0,0.5416,0.0,0.0,0.0
1,Abi Gharaq,2001-02-28,3094.206464,0.309421,6291.0,0.6291,0.0,0.0,0.0
2,Abi Gharaq,2001-03-31,3275.809481,0.327581,6513.0,0.6513,0.0,0.0,0.0
3,Abi Gharaq,2001-04-30,2596.503366,0.259650,5332.0,0.5332,0.0,0.0,0.0
4,Abi Gharaq,2001-05-31,2161.217474,0.216122,5147.0,0.5147,0.0,0.0,0.0


In [43]:
# no duplicates at this point
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['date','ADM3_EN'],keep=False)]

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT


In [44]:
# create 5-year interval fields to join with population data
adm3_iraq_data['year'] = adm3_iraq_data['date'].dt.year
adm3_iraq_data['year_5yr'] = np.where(
                               adm3_iraq_data['year'] < 2005, 2000,
                               np.where(adm3_iraq_data['year'] < 2010, 2005,
                                np.where(adm3_iraq_data['year'] < 2015, 2010,
                                np.where(adm3_iraq_data['year'] < 2020, 2015,
                                np.where(adm3_iraq_data['year'] < 2025, 2020, 0)))))
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,year,year_5yr
0,Abi Gharaq,2001-01-31,2886.373490,0.288637,5416.0,0.5416,0.0,0.0,0.0,2001,2000
1,Abi Gharaq,2001-02-28,3094.206464,0.309421,6291.0,0.6291,0.0,0.0,0.0,2001,2000
2,Abi Gharaq,2001-03-31,3275.809481,0.327581,6513.0,0.6513,0.0,0.0,0.0,2001,2000
3,Abi Gharaq,2001-04-30,2596.503366,0.259650,5332.0,0.5332,0.0,0.0,0.0,2001,2000
4,Abi Gharaq,2001-05-31,2161.217474,0.216122,5147.0,0.5147,0.0,0.0,0.0,2001,2000


In [45]:
# no duplicates at this point
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['date','ADM3_EN'],keep=False)]

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,year,year_5yr


In [52]:
# join population data by ADM3_PCODE and 5-year interval
pop_df_join = pop_df_full.drop(columns=['Shape_Area'])
print(adm3_iraq_data.shape)
adm3_iraq_data = adm3_iraq_data.merge(pop_df_join, how='outer',left_on=['ADM3_EN','year_5yr'],
                                      right_on=['ADM3_EN','year'])
adm3_iraq_data = adm3_iraq_data.drop(columns=['year_x','year_y','year_5yr'])
print(pop_df_join.shape)
print(adm3_iraq_data.shape)

(77616, 11)
(1470, 4)
(77616, 11)


In [53]:
# no duplicates at this point
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['date','ADM3_EN'],keep=False)]

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,pop_count,pop_density


In [54]:
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,pop_count,pop_density
0,Abi Gharaq,2001-01-31,2886.373490,0.288637,5416.0,0.5416,0.0,0.0,0.0,137199.66,8.709119e+06
1,Abi Gharaq,2001-02-28,3094.206464,0.309421,6291.0,0.6291,0.0,0.0,0.0,137199.66,8.709119e+06
2,Abi Gharaq,2001-03-31,3275.809481,0.327581,6513.0,0.6513,0.0,0.0,0.0,137199.66,8.709119e+06
3,Abi Gharaq,2001-04-30,2596.503366,0.259650,5332.0,0.5332,0.0,0.0,0.0,137199.66,8.709119e+06
4,Abi Gharaq,2001-05-31,2161.217474,0.216122,5147.0,0.5147,0.0,0.0,0.0,137199.66,8.709119e+06


#### IOM Data Preparation

In [60]:
# format IOM data for joining
iom_data_filt = iom_data_filt.rename(columns={
                                    'Q1.10 IsLocationInDisputedArea':'disputed_area',
                                    'Q1.11 WasLocationOccupied':'loc_occupied',
                                    'Q1.11.1 RetakenPeriod':'retaken_period',
                                    'Q1.12 FirstMajorEthno-religiousPrior':'maj_ethrelig_prior',
                                    'Q1.13 FirstMajorEthno-religiousCurrent':'maj_ethrelig_curr'         
                                    })

In [62]:
# add flags for whether attacked, occupied, or no ISIL activity
iom_data_filt['iom_attacked'] = np.where(iom_data_filt['loc_occupied']=='Yes, attacked', 1, 0)
iom_data_filt['iom_occupied'] = np.where(iom_data_filt['loc_occupied']=='Yes, occupied', 1, 0)
iom_data_filt['iom_no_isil_action'] = np.where(iom_data_filt['loc_occupied']=='No', 1, 0)
# add flags for whether in ISIL religious-ethnic group prior to crisis and currently
iom_data_filt['isil_ingroup_prior'] = np.where(iom_data_filt['maj_ethrelig_prior']=='Arab Sunni Muslim', 1, 0)
iom_data_filt['isil_ingroup_curr'] = np.where(iom_data_filt['maj_ethrelig_curr']=='Arab Sunni Muslim', 1, 0)
# convert disputed area answers to 1,0 flags
iom_data_filt['disputed_area'] = np.where(iom_data_filt['disputed_area']=='Yes', 1, 0)

In [63]:
# add flag for whether retaken pre- or post-2016 as indicator for long or short-term occupation
iom_data_filt['retaken_year'] = iom_data_filt['retaken_period'].str.extract("(\d\d\d\d)")
iom_data_filt['retaken_year'] = iom_data_filt['retaken_year'].astype('float')
iom_data_filt['retaken_post_2016'] = np.where(iom_data_filt['retaken_year'].isna(),iom_data_filt['retaken_year'],
                                              np.where(iom_data_filt['retaken_year']>=2017,1,0))

In [64]:
# drop unnecessary columns
iom_data_filt = iom_data_filt.drop(columns=['loc_occupied','retaken_period','maj_ethrelig_prior',
                                            'maj_ethrelig_curr','DateofInterview'])

In [69]:
iom_adm3_data = iom_data_filt.merge(iom_adm3_map.loc[:,('LocationID','ADM3_EN')],how='left',
                                    on='LocationID')
print(iom_data_filt.shape)
print(iom_adm3_data.shape)

(3717, 9)
(3717, 10)


In [70]:
# to aggregate, want to take the max of all fields except for iom_no_isil_action, which we want to take the
# min of - if any part of the adm3 area had isil action, we don't want to flag it as not having any action
iom_adm3_data_for_min = iom_adm3_data.loc[:,('ADM3_EN','iom_no_isil_action')]
iom_adm3_data_for_max = iom_adm3_data.drop(columns=['LocationID','iom_no_isil_action'])

In [71]:
iom_adm3_data_max_gb = iom_adm3_data_for_max.groupby(['ADM3_EN']).max().reset_index()
iom_adm3_data_min_gb = iom_adm3_data_for_min.groupby(['ADM3_EN']).min().reset_index()

In [72]:
iom_adm3_data_fin = iom_adm3_data_max_gb.merge(iom_adm3_data_min_gb,how='outer',
                                               on='ADM3_EN')
print(iom_adm3_data_max_gb.shape)
print(iom_adm3_data_min_gb.shape)
print(iom_adm3_data_fin.shape)

(215, 8)
(215, 2)
(215, 9)


In [73]:
# add IOM data to full dataset
print(adm3_iraq_data.shape)
adm3_iraq_data = adm3_iraq_data.merge(iom_adm3_data_fin, how='outer',
                                      on='ADM3_EN')
print(adm3_iraq_data.shape)

(77616, 11)
(77616, 19)


In [74]:
# identify number of Adm3 areas with no IOM data
len(adm3_iraq_data[adm3_iraq_data['iom_occupied'].isna()].ADM3_EN.unique())

79

In [75]:
adm3_iraq_data

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,pop_count,pop_density,disputed_area,iom_attacked,iom_occupied,isil_ingroup_prior,isil_ingroup_curr,retaken_year,retaken_post_2016,iom_no_isil_action
0,Abi Gharaq,2001-01-31,2886.373490,0.288637,5416.0,0.5416,0.0,0.0,0.0,137199.66,8.709119e+06,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0
1,Abi Gharaq,2001-02-28,3094.206464,0.309421,6291.0,0.6291,0.0,0.0,0.0,137199.66,8.709119e+06,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0
2,Abi Gharaq,2001-03-31,3275.809481,0.327581,6513.0,0.6513,0.0,0.0,0.0,137199.66,8.709119e+06,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0
3,Abi Gharaq,2001-04-30,2596.503366,0.259650,5332.0,0.5332,0.0,0.0,0.0,137199.66,8.709119e+06,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0
4,Abi Gharaq,2001-05-31,2161.217474,0.216122,5147.0,0.5147,0.0,0.0,0.0,137199.66,8.709119e+06,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77611,Zummar,2022-08-31,1032.033386,0.103203,5141.0,0.5141,NaN,NaN,NaN,185270.98,1.361171e+06,1.0,1.0,1.0,1.0,1.0,2015.0,0.0,0.0
77612,Zummar,2022-09-30,989.919904,0.098992,6549.0,0.6549,NaN,NaN,NaN,185270.98,1.361171e+06,1.0,1.0,1.0,1.0,1.0,2015.0,0.0,0.0
77613,Zummar,2022-10-31,973.940064,0.097394,5766.0,0.5766,NaN,NaN,NaN,185270.98,1.361171e+06,1.0,1.0,1.0,1.0,1.0,2015.0,0.0,0.0
77614,Zummar,2022-11-30,929.013765,0.092901,5069.0,0.5069,NaN,NaN,NaN,185270.98,1.361171e+06,1.0,1.0,1.0,1.0,1.0,2015.0,0.0,0.0


In [76]:
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['ADM3_EN','date'],keep=False)]

,ADM3_EN,date,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,pop_count,pop_density,disputed_area,iom_attacked,iom_occupied,isil_ingroup_prior,isil_ingroup_curr,retaken_year,retaken_post_2016,iom_no_isil_action


In [77]:
iraq_shp_adm3_filt = iraq_shp_adm3.loc[:,('Shape_Area', 'ADM3_EN', 'ADM3_AR', 'ADM3_PCODE',
       'ADM2_EN', 'ADM2_AR', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_AR', 'ADM1_PCODE',
       'geometry')]
iraq_shp_adm3[iraq_shp_adm3.duplicated(subset=['ADM3_EN'],keep=False)]

,Shape_Leng,Shape_Area,ADM3_EN,ADM3_AR,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM3ALT1AR,ADM3ALT2AR,...,ADM1_EN,ADM1_AR,ADM1_PCODE,ADM0_EN,ADM0_AR,ADM0_PCODE,date,validOn,validTo,geometry


In [78]:
# attach Adm3 geometries and parent areas to final dataset
iraq_shp_adm3_filt = iraq_shp_adm3.loc[:,('Shape_Area', 'ADM3_EN', 'ADM3_AR', 'ADM3_PCODE',
       'ADM2_EN', 'ADM2_AR', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_AR', 'ADM1_PCODE',
       'geometry')]
print(adm3_iraq_data.shape)
adm3_iraq_data_fin = adm3_iraq_data.merge(iraq_shp_adm3_filt, how='left',
                                          on='ADM3_EN')
adm3_iraq_data_fin = adm3_iraq_data_fin.rename(columns={'date':'month'})
print(adm3_iraq_data_fin.shape)

(77616, 19)
(77616, 29)


In [79]:
adm3_iraq_data_fin[adm3_iraq_data_fin.duplicated(subset=['month','ADM3_EN'],keep=False)]

,ADM3_EN,month,mean_evi,mean_evi_scaled,max_evi,max_evi_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,pop_count,...,Shape_Area,ADM3_AR,ADM3_PCODE,ADM2_EN,ADM2_AR,ADM2_PCODE,ADM1_EN,ADM1_AR,ADM1_PCODE,geometry


In [80]:
monthly_adm3_geo_file = source_dir + '/data/combined/monthly_adm3_data.geojson'
adm3_iraq_data_fin_gpd = gpd.GeoDataFrame(adm3_iraq_data_fin,geometry=adm3_iraq_data_fin['geometry'])
adm3_iraq_data_fin_gpd.to_file(monthly_adm3_geo_file,driver='GeoJSON')

In [ ]:
monthly_adm3_file = source_dir + '/data/combined/monthly_adm3_data.csv'
adm3_iraq_data_fin.drop(columns=['geometry']).to_csv(monthly_adm3_file,index=False,
                                                     encoding='utf-8-sig')